# Hoja de Trabajo: Análisis de Malware
### Javier Mombiela
### Carnet: 20067
### 23 de febrero del 2024

In [1]:
import pefile

#### Parte 1 – análisis estático

In [2]:
exe_path = './sample_vg655_25th.exe'

1. Utilice la herramienta pefile para examinar el PE header y obtenga las DLL y las APIs

In [3]:
def get_imports_info(exe_path):
    try:
        pe = pefile.PE(exe_path)
        print("DLLs importadas:")
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print("\t", entry.dll.decode())

        print("\nAPIs importadas:")
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print("\tDLL:", entry.dll.decode())
            for imp in entry.imports:
                print("\t\t", imp.name.decode())
    except Exception as e:
        print("Error:", e)

get_imports_info(exe_path)

DLLs importadas:
	 KERNEL32.dll
	 USER32.dll
	 ADVAPI32.dll
	 MSVCRT.dll

APIs importadas:
	DLL: KERNEL32.dll
		 GetFileAttributesW
		 GetFileSizeEx
		 CreateFileA
		 InitializeCriticalSection
		 DeleteCriticalSection
		 ReadFile
		 GetFileSize
		 WriteFile
		 LeaveCriticalSection
		 EnterCriticalSection
		 SetFileAttributesW
		 SetCurrentDirectoryW
		 CreateDirectoryW
		 GetTempPathW
		 GetWindowsDirectoryW
		 GetFileAttributesA
		 SizeofResource
		 LockResource
		 LoadResource
		 MultiByteToWideChar
		 Sleep
		 OpenMutexA
		 GetFullPathNameA
		 CopyFileA
		 GetModuleFileNameA
		 VirtualAlloc
		 VirtualFree
		 FreeLibrary
		 HeapAlloc
		 GetProcessHeap
		 GetModuleHandleA
		 SetLastError
		 VirtualProtect
		 IsBadReadPtr
		 HeapFree
		 SystemTimeToFileTime
		 LocalFileTimeToFileTime
		 CreateDirectoryA
		 GetStartupInfoA
		 SetFilePointer
		 SetFileTime
		 GetComputerNameW
		 GetCurrentDirectoryA
		 SetCurrentDirectoryA
		 GlobalAlloc
		 LoadLibraryA
		 GetProcAddress
		 GlobalFree
		

- No hay indicios inmediatos de comportamiento sospechoso en la cantidad de DLLs importadas ni en las APIs llamadas. El número de DLLs importadas (cuatro) no es por sí mismo alarmante, y las APIs llamadas son funciones estándar del sistema operativo Windows.

2. Obtenga la información de las secciones del PE Header. ¿Qué significa que algunas secciones tengan como parte de su nombre “upx”? Realice el procedimiento de desempaquetado para obtener las llamadas completas de las APIs.

- Las secciones con nombres que contienen "upx" generalmente indican que el archivo ha sido comprimido o empaquetado utilizando la herramienta de compresión UPX (Ultimate Packer for eXecutables).

In [4]:
def get_sections_info(exe_path):
    try:
        pe = pefile.PE(exe_path)
        print("Secciones del PE Header:")
        for section in pe.sections:
            print("\tNombre:", section.Name.decode().strip('\x00'))
            print("\tVirtualAddress:", hex(section.VirtualAddress))
            print("\tVirtualSize:", hex(section.Misc_VirtualSize))
            print("\tSizeOfRawData:", hex(section.SizeOfRawData))
            print("\tPointerToRawData:", hex(section.PointerToRawData))
            print("\tCharacteristics:", hex(section.Characteristics))
            print()
            
            # Desempaquetar secciones con "upx" en el nombre
            if b"upx" in section.Name.lower():
                print("Desempaquetando sección:", section.Name.decode().strip('\x00'))
                unpacked_data = section.get_data()
                print("Sección desempaquetada:", unpacked_data)
                
    except Exception as e:
        print("Error:", e)

get_sections_info(exe_path)

Secciones del PE Header:
	Nombre: .text
	VirtualAddress: 0x1000
	VirtualSize: 0x69b0
	SizeOfRawData: 0x7000
	PointerToRawData: 0x1000
	Characteristics: 0x60000020

	Nombre: .rdata
	VirtualAddress: 0x8000
	VirtualSize: 0x5f70
	SizeOfRawData: 0x6000
	PointerToRawData: 0x8000
	Characteristics: 0x40000040

	Nombre: .data
	VirtualAddress: 0xe000
	VirtualSize: 0x1958
	SizeOfRawData: 0x2000
	PointerToRawData: 0xe000
	Characteristics: 0xc0000040

	Nombre: .rsrc
	VirtualAddress: 0x10000
	VirtualSize: 0x349fa0
	SizeOfRawData: 0x34a000
	PointerToRawData: 0x10000
	Characteristics: 0x40000040



3. Según el paper “Towards Understanding Malware Behaviour by the Extraction of API Calls”, ¿en que categoría sospechosas pueden clasificarse estos ejemplos en base a algunas de las llamadas a las APIs que realizan? Muestre una tabla con las APIs sospechosas y la categoría de malware que el artículo propone